### Name : Harsh Sanjay Agrawal
### CWID : 10475285
### email: hagrawa1@stevens.edu
 (Fill in for 5pts)

##### **Collaboration Policy.** Homeworks will be done individually: each student must hand in their own answers. Use of partial or entire solutions obtained from others or online is strictly prohibited.

##### **Late Policy.** No late submissions will be allowed without consent from the instructor. If urgent or unusual circumstances prohibit you from submitting a homework assignment in time, please e-mail Professor or TAs explaining the situation.

##### **Submission format** : Complete this jupyter notebook to successfully implement recommender systems on the Movie Lens Dataset. Follow this link https://stackoverflow.com/questions/53460051/convert-ipynb-notebook-to-html-in-google-colab to convert your **executed** notebook and submit in the form of  html file. (We will need you to fill in all the required cells, run the cell to get the required output, and then convert the notebook to a HTML file)

#### Objective of the assignment : **Build a recommender system using the MovieLens Dataset** 
We have outlined the major steps involved to help you achieve the goal of building your own recommender system. Given a name of the movie, your model should output the top 5 movies similar to that particular movie.

1.   Import all the required libraries 
2.   Read the dataset
3.   Create a rating  matrix
4.   Normalise this rating matrix 
5.   Compute SVD of this normalised rating matrix
6.   Define a function that computes the cosine similarity between two movies
7.   Based on cosine similarity, sort the movies which are most similar and return the top 5 movies that match a given a movie title



In [1]:
# Import all the required libraries
import numpy as np
import pandas as pd

#### Read the Dataset using read_csv function from Pandas

In [2]:
## Read the dataset from the two files into ratings_data and movies_data
ratings_data  = pd.read_table("Data/ratings.dat" , header = None, sep = "::")
movies_data = pd.read_table("Data/movies.dat" , header = None, sep = "::")
user_data = pd.read_table("Data/users.dat" , header = None, sep = "::")

<ipython-input-2-89dd6de3ca48>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_data  = pd.read_table("Data/ratings.dat" , header = None, sep = "::")
<ipython-input-2-89dd6de3ca48>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_data = pd.read_table("Data/movies.dat" , header = None, sep = "::")
<ipython-input-2-89dd6de3ca48>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  user_data = pd.read_ta

In [3]:
ratings_data = ratings_data.rename(columns={0:"user_id" , 1 : "movie_id", 2:"rating", 3:"time"})
ratings_data.head()

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# UserID::Gender::Age::Occupation::Zip-code
user_data = user_data.rename(columns={0:"user_id" , 1 : "gender", 2:"age", 3:"occupation", 4:"zip_code"})
user_data.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
# MovieID::Title::Genres
movies_data = movies_data.rename(columns = {0:"movie_id",1:"title",2:"genre"})
movies_data.tail()

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [6]:
# Create an appropriate rating matrix where the rows of the matrix are movies and columns as users

ratings_mat = np.ndarray(shape=(np.max(ratings_data.movie_id.values), np.max(ratings_data.user_id.values)),dtype=np.uint8)
ratings_mat[ratings_data.movie_id.values-1, ratings_data.user_id.values-1] = ratings_data.rating.values
ratings_mat

array([[5, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [7]:
# Normalise the rating matrix and call it normalised_matrix
normalised_matrix = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T
normalised_matrix

array([[ 3.57400662, -1.42599338, -1.42599338, ..., -1.42599338,
        -1.42599338,  1.57400662],
       [-0.37152318, -0.37152318, -0.37152318, ..., -0.37152318,
        -0.37152318, -0.37152318],
       [-0.23874172, -0.23874172, -0.23874172, ..., -0.23874172,
        -0.23874172, -0.23874172],
       ...,
       [-0.03278146, -0.03278146, -0.03278146, ..., -0.03278146,
        -0.03278146, -0.03278146],
       [-0.02582781, -0.02582781, -0.02582781, ..., -0.02582781,
        -0.02582781, -0.02582781],
       [-0.24288079, -0.24288079, -0.24288079, ..., -0.24288079,
        -0.24288079, -0.24288079]])

In [8]:
# Compute the SVD of the normalised matrix. Use the svd function from np.linalg

U,S,V = np.linalg.svd(normalised_matrix.T / np.sqrt(ratings_mat.shape[0] - 1))
U

array([[ 7.13393053e-03,  1.64099327e-03,  2.14622406e-03, ...,
         4.52266370e-03,  6.00783778e-03,  8.85674875e-03],
       [ 6.40383513e-04, -2.70126226e-03, -2.00478360e-04, ...,
         2.54577717e-03,  5.40385541e-04, -1.55791869e-02],
       [ 6.72473390e-03, -3.34737240e-03, -3.95617989e-03, ...,
         1.71114905e-03,  5.91124462e-03,  8.44640940e-03],
       ...,
       [ 1.13666709e-02,  1.80896437e-03, -5.62198713e-04, ...,
         7.73909103e-01, -1.48780057e-02, -2.19959777e-03],
       [ 3.49381899e-03,  1.87620989e-02, -1.08962191e-02, ...,
        -9.78238175e-03,  3.32703088e-01, -3.14995403e-03],
       [-1.32856412e-02,  4.08015550e-02, -3.63311909e-03, ...,
        -1.09584563e-03,  5.44124149e-03,  8.83627250e-02]])

#### Cosine Similarity : Compute the cosine similarity to find the similarity between any two movies. Mathematically the cosine similarity is given as follows: 
$$ cosine(x,y) = \frac{x\cdot y^T}{||x||\cdot ||y||}  $$

In [9]:
# A function to calculate the cosine similarity sorted by most similar and return the top N results.

def cosine_similarity(ratings_data, movie_id, top=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = ratings_data[index, :]
    mag = np.sqrt(np.einsum('ij, ij -> i', ratings_data, ratings_data)) # Finding magnitute for denominator
    similar = np.dot(movie_row, ratings_data.T) / (mag[index] * mag) # Using the cosine formuka
    sort_indexes = np.argsort(-similar) # Sorting the array in order of similarity
    return sort_indexes[:top]

# A function to print top N similar movies

def print_top5_recommendations(movie_data, movie_id, top_indexes):
    print('Recommendations for the entered movie are:')
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])
        
# Initialise the value of id of the movie as given in the dataset, and number of top elements to be printed
sliced = V[:,:] # representative data

##### Get the top 5 recommendations given a title

In [10]:
# Enter 2808 for Universal Soldier

# movie_id = int(input("Enter a movie ID "))
movie_id = 2808
print()
indexes = cosine_similarity(sliced, movie_id, 6)
print_top5_recommendations(movies_data, movie_id, indexes)


Recommendations for the entered movie are:
Universal Soldier (1992)
Christmas Story, A (1983)
Universal Soldier: The Return (1999)
Problem Child 2 (1991)
Cat's Eye (1985)
Mickey Blue Eyes (1999)


# “I pledge my honor that I have abided by the Stevens Honor System.” 
